In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
! pip install scikit-optimize
# report scikit-optimize version number
import skopt
print('skopt %s' % skopt.__version__)
from skopt.space import Integer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding, Attention
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as K
import argparse

skopt 0.10.1


In [2]:
#data from kaggle dataset: "Prediction of music genre"
data1 = pd.read_csv("~/Downloads/music_genre.csv")
#data from kaggle data set: "Spotify Tracks Dataset"
data2 = pd.read_csv("~/Downloads/dataset.csv")

<h2>Data Cleaning for Data 2</h2>

In [3]:
#drop all the rows that are null
data2 = data2.dropna()

In [4]:
#make track_genre into label encoding
labelencoder = LabelEncoder()
data2['track_genre_num'] = labelencoder.fit_transform(data2['track_genre'])
data2

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,track_genre_num
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic,0
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic,0
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic,0
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic,0
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music,113
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music,113
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music,113
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music,113


In [5]:
features2 = ["popularity", "duration_ms", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"]
len(features2)

14

In [6]:
# train 80%, val 20%, test 20%
X_train2b, X_test2b, y_train2b, y_test2b = train_test_split(data2[features2], data2["track_genre_num"], test_size=0.2, random_state=1)
X_train2b, X_val2b, y_train2b, y_val2b = train_test_split(X_train2b, y_train2b, test_size=0.25, random_state=1)

In [7]:
model2b = Sequential()
model2b.add(LSTM(units=256,input_shape=(14, 1)))
model2b.add(Dense(114, activation = "softmax"))

model2b.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history2b = model2b.fit(X_train2b, y_train2b, validation_data=(X_val2b, y_val2b), batch_size=32, epochs=100)

Epoch 1/100
2138/2138 [==============================] - 91s 42ms/step - loss: 3.8788 - accuracy: 0.0842 - val_loss: 3.7018 - val_accuracy: 0.1010
Epoch 2/100
2138/2138 [==============================] - 86s 40ms/step - loss: 3.6406 - accuracy: 0.1096 - val_loss: 3.5762 - val_accuracy: 0.1198
Epoch 3/100
2138/2138 [==============================] - 79s 37ms/step - loss: 3.4575 - accuracy: 0.1416 - val_loss: 3.3444 - val_accuracy: 0.1675
Epoch 4/100
2138/2138 [==============================] - 79s 37ms/step - loss: 3.2387 - accuracy: 0.1822 - val_loss: 3.2227 - val_accuracy: 0.1879
Epoch 5/100
2138/2138 [==============================] - 82s 38ms/step - loss: 3.0622 - accuracy: 0.2135 - val_loss: 2.9754 - val_accuracy: 0.2339
Epoch 6/100
2138/2138 [==============================] - 76s 36ms/step - loss: 2.9003 - accuracy: 0.2457 - val_loss: 2.8945 - val_accuracy: 0.2494
Epoch 7/100
2138/2138 [==============================] - 87s 41ms/step - loss: 2.8158 - accuracy: 0.2609 - val_loss: 2

In [8]:
print("Training Accuracy:", model2b.evaluate(X_train2b, y_train2b))
y_test_pred2b = model2b.evaluate(X_test2b, y_test2b)
print("Test Accuracy:", y_test_pred2b)

2138/2138 [==============================] - 34s 16ms/step - loss: 0.5917 - accuracy: 0.7860
Training Accuracy: [0.5916812419891357, 0.7859764099121094]
713/713 [==============================] - 18s 25ms/step - loss: 4.7546 - accuracy: 0.2413
Test Accuracy: [4.75462532043457, 0.24127192795276642]


In [9]:
print(classification_report(y_test2b, np.argmax(model2b.predict(X_test2b, 1), axis =-1)))

22800/22800 [==============================] - 110s 5ms/step
              precision    recall  f1-score   support

           0       0.11      0.10      0.10       214
           1       0.24      0.22      0.23       202
           2       0.07      0.08      0.07       206
           3       0.12      0.14      0.13       201
           4       0.23      0.26      0.24       192
           5       0.11      0.09      0.10       199
           6       0.45      0.48      0.47       195
           7       0.31      0.37      0.33       178
           8       0.15      0.14      0.14       229
           9       0.11      0.09      0.10       206
          10       0.32      0.40      0.36       200
          11       0.07      0.07      0.07       187
          12       0.20      0.15      0.17       222
          13       0.41      0.36      0.39       203
          14       0.29      0.29      0.29       201
          15       0.10      0.09      0.09       208
          16       0

In [10]:
model2b2 = Sequential()
model2b2.add(LSTM(units=128,input_shape=(14, 1)))
model2b2.add(Dense(114, activation = "softmax"))

model2b2.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history2b2 = model2b2.fit(X_train2b, y_train2b, validation_data=(X_val2b, y_val2b), batch_size=32, epochs=100)

Epoch 1/100
2138/2138 [==============================] - 47s 20ms/step - loss: 3.9605 - accuracy: 0.0791 - val_loss: 3.7271 - val_accuracy: 0.0964
Epoch 2/100
2138/2138 [==============================] - 41s 19ms/step - loss: 3.6569 - accuracy: 0.1092 - val_loss: 3.6071 - val_accuracy: 0.1159
Epoch 3/100
2138/2138 [==============================] - 47s 22ms/step - loss: 3.5985 - accuracy: 0.1178 - val_loss: 3.5884 - val_accuracy: 0.1135
Epoch 4/100
2138/2138 [==============================] - 44s 21ms/step - loss: 3.4640 - accuracy: 0.1416 - val_loss: 3.3759 - val_accuracy: 0.1544
Epoch 5/100
2138/2138 [==============================] - 45s 21ms/step - loss: 3.2872 - accuracy: 0.1728 - val_loss: 3.2440 - val_accuracy: 0.1849
Epoch 6/100
2138/2138 [==============================] - 43s 20ms/step - loss: 3.1456 - accuracy: 0.2021 - val_loss: 3.1089 - val_accuracy: 0.2056
Epoch 7/100
2138/2138 [==============================] - 44s 21ms/step - loss: 3.0322 - accuracy: 0.2239 - val_loss: 2

In [19]:
print("Training Accuracy:", model2b2.evaluate(X_train2b, y_train2b))
y_test_pred2b2 = model2b2.evaluate(X_test2b, y_test2b)
print("Test Accuracy:", y_test_pred2b2)

2138/2138 [==============================] - 5s 2ms/step - loss: 1.6976 - accuracy: 0.4958
Training Accuracy: [1.6976228952407837, 0.49578210711479187]
713/713 [==============================] - 2s 3ms/step - loss: 3.1139 - accuracy: 0.2646
Test Accuracy: [3.1139473915100098, 0.26456141471862793]


In [11]:
scaler2 = preprocessing.MinMaxScaler()
scaled_feat2 = scaler2.fit_transform(data2[features2])

X_train2_2, X_test2_2, y_train2_2, y_test2_2 = train_test_split(scaled_feat2, data2["track_genre_num"], test_size=0.2, random_state=1)
X_train2_2, X_val2_2, y_train2_2, y_val2_2 = train_test_split(X_train2_2, y_train2_2, test_size=0.25, random_state=1)

In [12]:
model2_2 = Sequential()
#forget gates = 0, paththrough gate = 1
model2_2.add(LSTM(units=128,input_shape=(14, 1)))
model2_2.add(Dense(114, activation = "softmax"))

#bidirectional, attention block (inform importance of other positions)- layers.attention

model2_2.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history2_2 = model2_2.fit(X_train2_2, y_train2_2, validation_data=(X_val2_2, y_val2_2), batch_size=32, epochs=100)

Epoch 1/100
2138/2138 [==============================] - 31s 14ms/step - loss: 4.0507 - accuracy: 0.0662 - val_loss: 3.7472 - val_accuracy: 0.0937
Epoch 2/100
2138/2138 [==============================] - 25s 12ms/step - loss: 3.6707 - accuracy: 0.1076 - val_loss: 3.6163 - val_accuracy: 0.1191
Epoch 3/100
2138/2138 [==============================] - 31s 14ms/step - loss: 3.5477 - accuracy: 0.1294 - val_loss: 3.5683 - val_accuracy: 0.1320
Epoch 4/100
2138/2138 [==============================] - 27s 13ms/step - loss: 3.4463 - accuracy: 0.1464 - val_loss: 3.4154 - val_accuracy: 0.1469
Epoch 5/100
2138/2138 [==============================] - 26s 12ms/step - loss: 3.3660 - accuracy: 0.1581 - val_loss: 3.3160 - val_accuracy: 0.1661
Epoch 6/100
2138/2138 [==============================] - 28s 13ms/step - loss: 3.2929 - accuracy: 0.1717 - val_loss: 3.2595 - val_accuracy: 0.1802
Epoch 7/100
2138/2138 [==============================] - 37s 17ms/step - loss: 3.2129 - accuracy: 0.1857 - val_loss: 3

In [13]:
print("Training Accuracy:", model2_2.evaluate(X_train2_2, y_train2_2))
print("Test Accuracy:", model2_2.evaluate(X_test2_2, y_test2_2))

2138/2138 [==============================] - 4s 2ms/step - loss: 1.7511 - accuracy: 0.4836
Training Accuracy: [1.7510982751846313, 0.4836181700229645]
713/713 [==============================] - 2s 2ms/step - loss: 3.0486 - accuracy: 0.2733
Test Accuracy: [3.048637866973877, 0.2732894718647003]


In [14]:
model2_3 = Sequential()
#forget gates = 0, paththrough gate = 1

model2_3.add((Bidirectional(LSTM(units=128, input_shape=(14, 1)))))
model2_3.add(Dense(114, activation = "softmax"))

#bidirectional, attention block (inform importance of other positions)- layers.attention

model2_3.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history2_3 = model2_3.fit(np.expand_dims(X_train2_2, 1), y_train2_2, validation_data=(np.expand_dims(X_val2_2, 1), y_val2_2), batch_size=32, epochs=100)

Epoch 1/100
2138/2138 [==============================] - 8s 2ms/step - loss: 3.8647 - accuracy: 0.1213 - val_loss: 3.6175 - val_accuracy: 0.1511
Epoch 2/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.5876 - accuracy: 0.1590 - val_loss: 3.5474 - val_accuracy: 0.1663
Epoch 3/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.5258 - accuracy: 0.1712 - val_loss: 3.4787 - val_accuracy: 0.1770
Epoch 4/100
2138/2138 [==============================] - 4s 2ms/step - loss: 3.4322 - accuracy: 0.1876 - val_loss: 3.3675 - val_accuracy: 0.1961
Epoch 5/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.3121 - accuracy: 0.2028 - val_loss: 3.2511 - val_accuracy: 0.2110
Epoch 6/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.2368 - accuracy: 0.2138 - val_loss: 3.2087 - val_accuracy: 0.2153
Epoch 7/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.1935 - accuracy: 0.2180 - val_loss: 3.1715 - val_ac

In [15]:
print("Training Accuracy:", model2_3.evaluate(np.expand_dims(X_train2_2, 1), y_train2_2))
print("Test Accuracy:", model2_3.evaluate(np.expand_dims(X_test2_2, 1), y_test2_2))

2138/2138 [==============================] - 2s 743us/step - loss: 2.4271 - accuracy: 0.3472
Training Accuracy: [2.427096128463745, 0.3471542000770569]
713/713 [==============================] - 1s 722us/step - loss: 2.6346 - accuracy: 0.3065
Test Accuracy: [2.6345653533935547, 0.30653509497642517]


In [16]:
model2_4 = Sequential()
#forget gates = 0, paththrough gate = 1

model2_4.add(Bidirectional(LSTM(units=128, input_shape=(14, 1))))
model2_4.add(Dropout(0.1))
model2_4.add(Dense(114, activation = "softmax"))

#bidirectional, attention block (inform importance of other positions)- layers.attention

model2_4.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
history2_4 = model2_4.fit(np.expand_dims(X_train2_2, 1), y_train2_2, validation_data=(np.expand_dims(X_val2_2, 1), y_val2_2), batch_size=32, epochs=100)

Epoch 1/100
2138/2138 [==============================] - 8s 3ms/step - loss: 3.8814 - accuracy: 0.1191 - val_loss: 3.6228 - val_accuracy: 0.1545
Epoch 2/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.6009 - accuracy: 0.1576 - val_loss: 3.5594 - val_accuracy: 0.1664
Epoch 3/100
2138/2138 [==============================] - 6s 3ms/step - loss: 3.5458 - accuracy: 0.1692 - val_loss: 3.4942 - val_accuracy: 0.1764
Epoch 4/100
2138/2138 [==============================] - 5s 3ms/step - loss: 3.4590 - accuracy: 0.1800 - val_loss: 3.3771 - val_accuracy: 0.1941
Epoch 5/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.3412 - accuracy: 0.1963 - val_loss: 3.2665 - val_accuracy: 0.2104
Epoch 6/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.2636 - accuracy: 0.2088 - val_loss: 3.2176 - val_accuracy: 0.2124
Epoch 7/100
2138/2138 [==============================] - 5s 2ms/step - loss: 3.2190 - accuracy: 0.2145 - val_loss: 3.1791 - val_ac

In [17]:
print("Training Accuracy:", model2_4.evaluate(np.expand_dims(X_train2_2, 1), y_train2_2))
y_test_pred2_4 = model2_4.evaluate(np.expand_dims(X_test2_2, 1), y_test2_2)
print("Test Accuracy:", y_test_pred2_4)

2138/2138 [==============================] - 2s 860us/step - loss: 2.4882 - accuracy: 0.3381
Training Accuracy: [2.488222122192383, 0.3381189703941345]
713/713 [==============================] - 1s 1ms/step - loss: 2.6406 - accuracy: 0.3028
Test Accuracy: [2.640625238418579, 0.3027631640434265]


In [18]:
print(classification_report(y_test2_2, np.argmax(model2_4.predict(np.expand_dims(X_test2_2, 1)), axis =-1)))

713/713 [==============================] - 1s 711us/step
              precision    recall  f1-score   support

           0       0.19      0.15      0.17       214
           1       0.32      0.31      0.31       202
           2       0.12      0.04      0.06       206
           3       0.12      0.07      0.09       201
           4       0.31      0.40      0.35       192
           5       0.14      0.10      0.12       199
           6       0.59      0.51      0.55       195
           7       0.44      0.53      0.48       178
           8       0.11      0.02      0.04       229
           9       0.16      0.29      0.21       206
          10       0.37      0.46      0.41       200
          11       0.07      0.03      0.04       187
          12       0.25      0.20      0.22       222
          13       0.49      0.52      0.50       203
          14       0.34      0.38      0.36       201
          15       0.23      0.19      0.21       208
          16       0.51 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3da032f1-e5ab-4726-ac09-eb3d9c053730' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>